# Candidate alignment (merging) 

**Necessary files:**

 - event_cands = dataframe with candidates and their meta information
 - model = embeddings model trained on the event dataset (for the second merging step)
 

In [1]:
from merging_steps import merging_step1, merging_step2, merging_step3, merging_step4
from merging_steps import merge_indices, merge_transitively
import gensim  
import pandas as pd

import pickle
import os
import re
from tqdm import tqdm
from collections import defaultdict


# functions pickle_file and load_pickle merely help with storing pickled files in the event folders on drive
def pickle_file(file_name, file_to_dump):
    directory_path = os.getcwd() + "/../../../../"
    folder_name = file_name.split('_')[0]
    file_path = directory_path +  fr"Dropbox (CBS)/Master thesis data/Candidate Data/{folder_name}/{file_name}"
    with open(file_path, 'wb') as fp:
        pickle.dump(file_to_dump, fp)

def load_pickle(file_name):
    directory_path = os.getcwd() + "/../../../../"
    folder_name = file_name.split('_')[0]
    file_path = directory_path + fr"Dropbox (CBS)/Master thesis data/Candidate Data/{folder_name}/{file_name}"
    with open(file_path, "rb") as input_file:
        return pickle.load(input_file)

In [25]:
#event candidates can be accessed using load_pickle function under the name [event]_short_cands
event_cands = load_pickle('rohingya_cands')
sim_df

(52554, 8)

In [26]:
event_cands

,candidates,cand_tags,cand_text,cand_head,cand_type,cand_len,cand_freq,string_len
0,"(refugees, refugees, misc)","[\n {\n ""id"": 1,\n ""text"": ""refugees"",\...",refugees,refugees,misc,1,2483,8
1,"(Myanmar, Myanmar, loc-ne)","[\n {\n ""id"": 1,\n ""text"": ""Myanmar"",\n...",myanmar,myanmar,loc-ne,1,2331,7
2,"(Thailand, Thailand, loc-ne)","[\n {\n ""id"": 1,\n ""text"": ""Thailand"",\...",thailand,thailand,loc-ne,1,1486,8
3,"(Bangladesh, Bangladesh, loc-ne)","[\n {\n ""id"": 1,\n ""text"": ""Bangladesh""...",bangladesh,bangladesh,loc-ne,1,1475,10
4,"(myanmar, myanmar, misc)","[\n {\n ""id"": 1,\n ""text"": ""myanmar"",\n...",myanmar,myanmar,misc,1,1252,7
...,...,...,...,...,...,...,...,...
52549,"(Ros, Ros, misc)","[\n {\n ""id"": 1,\n ""text"": ""Ros"",\n ...",ros,ros,misc,1,1,3
52550,"(Myanmar and nothing, Myanmar, loc-ne)","[\n {\n ""id"": 1,\n ""text"": ""Myanmar"",\n...",myanmar and nothing,myanmar,loc-ne,3,1,19
52551,"(Iraq and Libya, Iraq, loc-ne)","[\n {\n ""id"": 1,\n ""text"": ""Iraq"",\n ...",iraq and libya,iraq,loc-ne,3,1,14
52552,"(wealth to exploit, wealth, misc)","[\n {\n ""id"": 1,\n ""text"": ""wealth"",\n ...",wealth to exploit,wealth,misc,3,1,17


### Merging step 1
  - In the first merging step, we merge two candidates if the head of each of their representative phrase is identical by string comparison. Therefore, we merge candidates if the cand_head column value is equal. We keep the first row from top and sum the cand_freq column to keep count of all merged candidates.
  - sort by frequency after grouping


In [2]:

def merging_step1(event_cands):
    event_cands_merged = event_cands.groupby(['cand_head'],sort=False,as_index=False).agg({
        'candidates': 'first', 
        'cand_text': 'first',
        'cand_type': 'first',
        'cand_freq':'sum'}).sort_values(by=['cand_freq'],ascending=False).reset_index(drop=True)
    print(f'there are {len(event_cands_merged)} candidates after 1st merging step')
    return event_cands_merged

In [4]:
#rohingya
event_cands = load_pickle('rohingya_cands')
event_cands_merged = merging_step1(event_cands)
pickle_file('rohingya_cands_after1', event_cands_merged)

In [5]:
#tigray
event_cands = load_pickle('tigray_cands')
event_cands_merged = merging_step1(event_cands)
pickle_file('tigray_cands_after1', event_cands_merged)

In [6]:
#channel
event_cands = load_pickle('channel_cands')
event_cands_merged = merging_step1(event_cands)
pickle_file('channel_cands_after1', event_cands_merged)

## Additional merging attempts 2: Similarity using BERT embeddings

In [7]:

def merging_bert_len(event_cands, sim_df, low_threshold = 0.8, high_threshold = 0.9):
    what_merged = defaultdict(list)
    merged_cands = set()
    
    #merge the dataframes so we have information about frequencies of candidates in sim_df
    # use outer join to include the candidates that are not similar to any other candidate (thus not in sim_df)
    merged_df = pd.merge(sim_df,event_cands, how='outer', left_on='text_to_compare', right_on='cand_text')
    merged_df = merged_df[~merged_df['cand_freq'].isnull()] # if there are any extra records in sim_df, remove them
    merged_df.text.fillna(merged_df.cand_text, inplace=True) 
    
    #in general, short words tend to be similar to much more words than long ones, setting threshold for length can help
    merged_df['cand_len'] = merged_df['text'].apply(lambda cand: max([len(word) for word in cand.split()]))
    #merged_df['cand_freq'] = pd.to_numeric(merged_df['cand_freq'], downcast='integer')
    ls = merged_df.text.unique()
    for cand in tqdm(ls):
        #start = time()
        if cand not in merged_cands:
            #cp1 = time()-start
            #print(f'first checkpoint: {cp1}s')
            # apply threshold of 5 letters to apply different similarities to match
            lookup_df_long = merged_df[(merged_df.text==cand) & (merged_df.sim>low_threshold) & (merged_df.cand_len>5)]
            lookup_df_short = merged_df[(merged_df.text==cand) & (merged_df.sim>high_threshold) & (merged_df.cand_len<=5)]
            lookup_df = pd.concat([lookup_df_long, lookup_df_short],ignore_index=True)
            #cp2 = time()-cp1
            #print(f'second checkpoint: {cp1}s')
            #print(cand)
            #print(list(lookup_df['candidates']))
        
            what_merged[cand].extend(list(lookup_df['candidates']))
            #cp3 = time()-cp2
            #print(f'third checkpoint: {cp3}s')
        merged_cands.update(list(lookup_df['text_to_compare']))
        #print(f'second checkpoint: {cp1}s')
    
    #clean nans from the dictionary
    for key in what_merged.keys():
        what_merged[key] = [value for value in what_merged[key] if value != 'nan']
    
    merged_df_long = merged_df[((merged_df['sim']>low_threshold) | (merged_df['sim'].isnull())) & (merged_df['cand_len']>5)]
    merged_df_short = merged_df[((merged_df['sim']>high_threshold) | (merged_df['sim'].isnull())) & (merged_df['cand_len']<=5)]
    
    final_cands_long = merged_df_long.groupby(['text'],sort=False,as_index=False).agg({'cand_freq':'sum'})
    final_cands_short = merged_df_short.groupby(['text'],sort=False,as_index=False).agg({'cand_freq':'sum'})
    #print(final_cands_long)
    #print(final_cands_short)
    event_cands_final = pd.concat([final_cands_long, final_cands_short],ignore_index=True)    
    event_cands_final = event_cands_final[~event_cands_final['text'].isin(merged_cands)]
    event_cands_final = event_cands_final.sort_values(by=['cand_freq'],ascending=False)
    event_cands_final.reset_index(drop=True, inplace=True)
    
    return event_cands_final, what_merged
 


In [80]:
# second merging step for rohingya
sim_df = load_pickle('rohingya_sent_sim_df')
event_cands = load_pickle('rohingya_cands_after1')

In [96]:
    def get_merged_candidates(cand, low_threshold = low_threshold, high_threshold = high_threshold):
            lookup_df = pd.DataFrame({'candidates': [], 'text_to_compare':[]})
            if cand not in merged_cands:
                lookup_df_full = merged_df_small[merged_df_small.text==cand]
                if method == 'len':
                    # use bound of 5 letters for different thresholds to be applied
                    lookup_df_long = lookup_df_full[lookup_df_full.cand_len>5]
                    lookup_df_short = lookup_df_full[(lookup_df_full.sim>high_threshold) & 
                                                      (lookup_df_full.cand_len<=5)]                  
                else:
                    # apply misc to apply different similarities to match
                    lookup_df_long = lookup_df_full[lookup_df_full.cand_type!='misc']
                    lookup_df_short = lookup_df_full[(lookup_df_full.sim>high_threshold) & 
                                                      (lookup_df_full.cand_type=='misc')] 
                    
                lookup_df = pd.concat([lookup_df_long, lookup_df_short],ignore_index=True)
                #what_merged[cand].extend(list(lookup_df['candidates']))
                merged_cands.update(list(lookup_df['text_to_compare']))

            return list(lookup_df['candidates'])

In [ ]:
#%load_ext line_profiler
method='misc'
merged_df_small = merged_df[merged_df.sim>low_threshold]


In [97]:
%lprun -f get_merged_candidates get_merged_candidates("refugees")

In [105]:
def merging_step2( event_cands, sim_df,low_threshold = 0.8, high_threshold = 0.9, method='misc'):

    what_merged = defaultdict(list)
    merged_cands = set()

    #merge the dataframes so we have information about frequencies of candidates in sim_df
    # use outer join to include the candidates that are not similar to any other candidate (thus not in sim_df)
    merged_df = pd.merge(sim_df,event_cands, how='outer', left_on='text_to_compare', right_on='cand_text')
    merged_df = merged_df[~merged_df['cand_freq'].isnull()] # if there are any extra records in sim_df, remove them
    merged_df.text.fillna(merged_df.cand_text, inplace=True) 

    tqdm.pandas() 
    #in general, short words tend to be similar to much more words than long ones, setting threshold for length can help
    merged_df['cand_len'] = merged_df['text'].progress_apply(lambda cand: max([len(word) for word in cand.split()]))
    # and select only the rows above the lower threshold, so we do not have to filter by it in the loop
    merged_df_small = merged_df[merged_df.sim>low_threshold]
    
    # create an object to iterate over unique candidates
    unique_merged_df = merged_df.groupby(by=['text'], sort = False, as_index=False).agg({'cand_len':'first'})
    
    def get_merged_candidates(cand, low_threshold = low_threshold, high_threshold = high_threshold):
            lookup_df = pd.DataFrame({'candidates': [], 'text_to_compare':[]})
            if cand not in merged_cands:
                lookup_df_full = merged_df_small[merged_df_small.text==cand]
                if method == 'len':
                    # use bound of 5 letters for different thresholds to be applied
                    lookup_df_long = lookup_df_full[lookup_df_full.cand_len>5]
                    lookup_df_short = lookup_df_full[(lookup_df_full.sim>high_threshold) & 
                                                      (lookup_df_full.cand_len<=5)]                  
                else:
                    # apply misc to apply different similarities to match
                    lookup_df_long = lookup_df_full[lookup_df_full.cand_type!='misc']
                    lookup_df_short = lookup_df_full[(lookup_df_full.sim>high_threshold) & 
                                                      (lookup_df_full.cand_type=='misc')] 
                    
                lookup_df = pd.concat([lookup_df_long, lookup_df_short],ignore_index=True)
                #what_merged[cand].extend(list(lookup_df['candidates']))
                merged_cands.update(list(lookup_df['text_to_compare']))

            return list(lookup_df['candidates'])

    unique_merged_df['merged'] =  unique_merged_df.text.progress_apply(get_merged_candidates) 

    merged_df_long = merged_df_small[ (merged_df_small['sim'].isnull()) & (merged_df_small['cand_len']>5)]
    merged_df_short = merged_df_small[((merged_df_small['sim']>high_threshold) | (merged_df_small['sim'].isnull())) & (merged_df_small['cand_len']<=5)]

    final_cands_long = merged_df_long.groupby(['text'],sort=False,as_index=False).agg({'cand_freq':'sum'})
    final_cands_short = merged_df_short.groupby(['text'],sort=False,as_index=False).agg({'cand_freq':'sum'})
    #print(final_cands_long)
    #print(final_cands_short)
    event_cands_final = pd.concat([final_cands_long, final_cands_short],ignore_index=True) 

    event_cands_final = event_cands_final[~event_cands_final['text'].isin(merged_cands)]
    event_cands_final = event_cands_final.sort_values(by=['cand_freq'],ascending=False)
    event_cands_final.reset_index(drop=True, inplace=True)

    for cand,merged_list in zip(unique_merged_df['text'],unique_merged_df['merged']):
        what_merged[cand] = merged_list
    
    return event_cands_final, what_merged


In [66]:
pickle_file('greece_final_cands',event_cands_final)
pickle_file('greece_what_merged',what_merged)

In [ ]:
#clean nans from the dictionary
for key in what_merged.keys():
    #print(what_merged[key])
    if what_merged[key]!= 'nan':
        what_merged[key] = [value for value in what_merged[key] if value != 'nan']  

In [55]:
what_merged['turkey']

[('Erdogan', 'Erdogan', 'person-ne'),
 ('Turks', 'Turks', 'person-ne'),
 ('Turkeys', 'Turkeys', 'misc'),
 ('Turkish', 'Turkish', 'person-ne'),
 ('Turkish soldiers', 'soldiers', 'misc'),
 ('the Turkish side', 'side', 'misc'),
 ('Turkish army', 'army', 'group-ne'),
 ('Turkish officials', 'officials', 'misc'),
 ('2 - Turkey', '2', 'misc'),
 ('Turkey hosts', 'hosts', 'misc'),
 ('the Turkish coast', 'coast', 'misc'),
 ('its agents in Turkey', 'agents', 'misc'),
 ('the defensive as Turkey', 'defensive', 'misc'),
 ('Turkish flags', 'flags', 'misc'),
 ('Turkish flag', 'flag', 'misc'),
 ('Ankaras', 'Ankaras', 'misc'),
 ('Turkish Presidency', 'Presidency', 'misc'),
 ('turkey anymore, there', 'anymore,', 'misc'),
 ('rnek 1 - Turkey', 'rnek', 'misc'),
 ('Turkish ambassador', 'ambassador', 'misc'),
 ('Turkish MFA', 'MFA', 'misc'),
 ('Turkish Airways', 'Airways', 'group-ne'),
 ('Turkeyborder', 'Turkeyborder', 'loc-ne'),
 ('Turkish columnist', 'columnist', 'misc'),
 ('Turkey opensborders', 'opensbord

In [37]:
#new_merged_df['merged'] =  new_merged_df[:100].text.progress_apply(merging_bert_len) 
merged_df[merged_df['text_to_compare']=='2 - Turkey']

,text,text_to_compare,sim,cand_head,candidates,cand_text,cand_type,cand_freq,cand_len


In [3]:
event_cands[event_cands.cand_text=='smarty']#[event_cands.cand_freq==30]

,cand_head,candidates,cand_text,cand_type,cand_freq
9695,smarty,"(Smarty, Smarty, misc)",smarty,misc,1


In [ ]:
merged_df_long[merged_df_long.text=='smarty']#[event_cands_final.cand_freq==30]

In [208]:
what_merged['smarty']

[]

In [104]:
sim_df

,text,text_to_compare,sim
0,refugees,karen refugees,0.762601
1,refugees,a rohingya refugee camp,0.740026
2,refugees,refugee camps,0.881408
3,refugees,refugee,0.950323
4,refugees,shelters,0.707489
...,...,...,...
4282924,eao in mon state,brahmanical,0.722714
4282925,eao in mon state,congis,0.735037
4282926,naive optimism,many innocent buddists,0.704713
4282927,wokologics,brahmanical,0.768744


In [ ]:
# second merging step for rohingya
sim_df = load_pickle('rohingya_sent_sim_df')
event_cands = load_pickle('rohingya_cands_after1')

event_cands_final, what_merged = merging_step2(event_cands, sim_df)
pickle_file('rohingya_final_cands',event_cands_final)
pickle_file('rohingya_what_merged',what_merged)

In [107]:
# second merging step for greece
sim_df = load_pickle('greece_sent_sim_df')
event_cands = load_pickle('greece_cands_after1')

event_cands_final, what_merged = merging_step2(event_cands, sim_df)
pickle_file('greece_final_cands2',event_cands_final)
pickle_file('greece_what_merged2',what_merged)

100%|██████████████████████████████████████████████████████████████████████████| 37940/37940 [5:17:49<00:00,  1.99it/s]


In [8]:
# second merging step for tigray
sim_df = load_pickle('tigray_sent_sim_df')
event_cands = load_pickle('tigray_cands_after1')

event_cands_final, what_merged = merging_step2(event_cands, sim_df)
pickle_file('tigray_final_cands',event_cands_final)
pickle_file('tigray_what_merged',what_merged)

100%|██████████████████████████████████████████████████████████████████████████| 10700/10700 [1:40:48<00:00,  1.77it/s]


In [9]:
event_cands_final

,text,cand_freq
0,access,17771.0
1,refugees,10096.0
2,tigray,8026.0
3,tigray genocide,2907.0
4,isaias afeworki,2135.0
...,...,...
2648,amhara enticement,1.0
2649,ns,1.0
2650,the blm participant,1.0
2651,candies,1.0


In [10]:
what_merged

defaultdict(list,
            {'tigray': [('Eritrea', 'Eritrea', 'loc-ne'),
              ('tigrayans', 'tigrayans', 'misc'),
              ('tigrai', 'tigrai', 'misc'),
              ('tigray eritrea out', 'out', 'misc'),
              ('Hamdayet', 'Hamdayet', 'loc-ne'),
              ('tigraygenocide', 'tigraygenocide', 'misc'),
              ('Amharic', 'Amharic', 'misc'),
              ('Yohannes Abraham tigray', 'Yohannes', 'person-ne'),
              ('Tigrays', 'Tigrays', 'misc'),
              ('Tigraway', 'Tigraway', 'loc-ne'),
              ('tigray demands', 'demands', 'misc'),
              ('Tigrayan origin', 'origin', 'misc'),
              ('Tigrayan getting', 'Tigrayan', 'loc-ne'),
              ('tigra', 'tigra', 'misc'),
              ('WPF tigrai', 'WPF', 'misc'),
              ('Tegray', 'Tegray', 'misc'),
              ('Tigay', 'Tigay', 'loc-ne'),
              ('Tigray Dev Association', 'Association', 'misc'),
              ('tigraymany', 'tigraymany', 'misc'),
 

In [16]:
def merging_bert_misc(event_cands, sim_df, low_threshold = 0.8, high_threshold = 0.9):
    what_merged = defaultdict(list)
    merged_cands = set()
    
    #merge the dataframes so we have information about frequencies of candidates in sim_df
    # use outer join to include the candidates that are not similar to any other candidate (thus not in sim_df)
    merged_df = pd.merge(sim_df,event_cands, how='outer', left_on='text_to_compare', right_on='cand_text')
    merged_df = merged_df[~merged_df['cand_freq'].isnull()] # if there are any extra records from sim_df, remove them
    merged_df.text.fillna(merged_df.cand_text, inplace=True) 
    
    for cand in tqdm(merged_df['text'].unique()):
        if cand not in merged_cands:
            # apply threshold of 5 letters to apply different similarities to match
            lookup_df_nonmisc = merged_df[(merged_df['text']==cand) & (merged_df['sim']>low_threshold) & (merged_df['cand_type']!='misc')]
            lookup_df_misc = merged_df[(merged_df['text']==cand) & (merged_df['sim']>high_threshold) & (merged_df['cand_type']=='misc')]
            lookup_df = pd.concat([lookup_df_nonmisc, lookup_df_misc],ignore_index=True)
            #print(cand)
            #print(list(lookup_df['candidates']))
        
            what_merged[cand].extend(list(lookup_df['candidates']))
        merged_cands.update(list(lookup_df['text_to_compare']))
    
    #clean nans from the dictionary
    for key in what_merged.keys():
        what_merged[key] = [value for value in what_merged[key] if value != 'nan']
    
    merged_df_nonmisc = merged_df[((merged_df['sim']>low_threshold) | (merged_df['sim'].isnull())) & (merged_df['cand_type']!='misc')]
    merged_df_misc = merged_df[((merged_df['sim']>high_threshold) | (merged_df['sim'].isnull())) & (merged_df['cand_type']=='misc')]
    
    final_cands_nonmisc = merged_df_nonmisc.groupby(['text'],sort=False,as_index=False).agg({'cand_freq':'sum'})
    final_cands_misc = merged_df_misc.groupby(['text'],sort=False,as_index=False).agg({'cand_freq':'sum'})
    #print(final_cands_long)
    #print(final_cands_short)
    event_cands_final = pd.concat([final_cands_nonmisc, final_cands_misc],ignore_index=True)
    
    #finally, we remove the rows with candidates that have been merged
    event_cands_final = event_cands_final[~event_cands_final['text'].isin(merged_cands)]
    event_cands_final = event_cands_final.sort_values(by=['cand_freq'],ascending=False)
    event_cands_final.reset_index(drop=True, inplace=True)
    
    return event_cands_final, what_merged

In [13]:
# second merging step for rohingya
sim_df = load_pickle('rohingya_sent_sim_df')
event_cands = load_pickle('rohingya_cands_after1')

event_cands_final, what_merged = merging_bert_misc(event_cands, sim_df)
pickle_file('rohingya_final_cands2',event_cands_final)
pickle_file('rohingya_what_merged2',what_merged)

100%|██████████████████████████████████████████████████████████████████████████| 10519/10519 [3:59:31<00:00,  1.37s/it]


In [14]:
# second merging step for rohingya
sim_df = load_pickle('tigray_sent_sim_df')
event_cands = load_pickle('tigray_cands_after1')

event_cands_final, what_merged = merging_bert_misc(event_cands, sim_df)
pickle_file('tigray_final_cands2',event_cands_final)
pickle_file('tigray_what_merged2',what_merged)

100%|██████████████████████████████████████████████████████████████████████████| 10700/10700 [3:01:46<00:00,  1.02s/it]


In [ ]:
# second merging step for rohingya
sim_df = load_pickle('greece_sent_sim_df')
event_cands = load_pickle('greece_cands_after1')

event_cands_final, what_merged = merging_bert_misc(event_cands, sim_df)
pickle_file('greece_final_cands2',event_cands_final)
pickle_file('greece_what_merged2',what_merged)

 17%|███████████▉                                                           | 6398/37940 [11:59:59<54:00:53,  6.16s/it]

In [5]:
#candidate_list = all unique candidates
#sim_df = dataframe with similarities of all candidate pairs (larger than 0.7)

def merging_step_bert(candidate_df, sim_df):
    
    indices_to_remove = set()
    what_merged= defaultdict(list)
    
    merged_df = pd.merge(candidate_df,sim_df, how='left', left_on='cand_text', right_on='text_to_compare')
    for cand_id in tqdm(range(len(candidate_df['cand_text']))): 
        candidate = candidate_df['candidates'][cand_id][0]
        is_merged = False
        for key in what_merged.keys():
            for value in what_merged[key]:
                if candidate == value[0]:
                    is_merged = True
                    indices_to_remove.add(cand_id)
                    continue
            if is_merged==True:
                continue
        if is_merged == True:
            continue
            print('I should not see this')
        lookup_merged_df = merged_df[(merged_df['sim']>0.8) & (merged_df['text']==str(candidate))] 
        what_merged[candidate].extend(list(merged_df['candidates']))

    return indices_to_remove, what_merged

ids, wm_bert = merging_step_bert(event_cands,sim_df)

100%|████████████████████████████████████████████████████████████████████████████| 10190/10190 [35:27<00:00,  4.79it/s]


In [6]:
event_cands_merged_bert = merge_indices(event_cands, ids)

Initial amount of candidates: 10190
Amount of candidates: 1, after removing 10189 indices


In [39]:
merged_df = pd.merge(event_cands,sim_df, how='left', left_on='cand_text', right_on='text')
merged_df[(merged_df['text']=='tigray')&(merged_df['sim']>0.8)]

for cand in tqdm(merged_df['cand_text']):
    #print(merged_df['text_to_compare'])
    if merged_df['text_to_compare'].eq(cand).any():
        merged_df = merged_df[merged_df['text_to_compare']!=cand]


        


  0%|                                                                        | 264/4244649 [01:20<360:58:45,  3.27it/s]


KeyboardInterrupt: 

In [37]:
merged_df

,candidates,cand_tags,cand_text,cand_len,cand_freq,string_len,text,text_to_compare,sim
0,"(tigray, tigray, {tigray}, misc)","[\n {\n ""id"": 1,\n ""text"": ""tigray"",\n ...",tigray,1,2899,6,tigray,ethiopia,0.726208
1,"(tigray, tigray, {tigray}, misc)","[\n {\n ""id"": 1,\n ""text"": ""tigray"",\n ...",tigray,1,2899,6,tigray,eritrea,0.803327
2,"(tigray, tigray, {tigray}, misc)","[\n {\n ""id"": 1,\n ""text"": ""tigray"",\n ...",tigray,1,2899,6,tigray,unhcr,0.715736
3,"(tigray, tigray, {tigray}, misc)","[\n {\n ""id"": 1,\n ""text"": ""tigray"",\n ...",tigray,1,2899,6,tigray,sudan,0.725412
4,"(tigray, tigray, {tigray}, misc)","[\n {\n ""id"": 1,\n ""text"": ""tigray"",\n ...",tigray,1,2899,6,tigray,tplf,0.720120
...,...,...,...,...,...,...,...,...,...
4244644,"(the wolle, wolle, {wolle}, misc)","[\n {\n ""id"": 1,\n ""text"": ""the"",\n ...",the wolle,2,1,9,the wolle,hrwde,0.842103
4244645,"(the wolle, wolle, {wolle}, misc)","[\n {\n ""id"": 1,\n ""text"": ""the"",\n ...",the wolle,2,1,9,the wolle,60660 tegaru,0.715475
4244646,"(hrwde, hrwde, {hrwde}, misc)","[\n {\n ""id"": 1,\n ""text"": ""hrw_de"",\n ...",hrwde,1,1,5,hrwde,60660 tegaru,0.762883
4244647,"( nelson mandela eritrea, nelson, {nelson, eri...","[\n {\n ""id"": 1,\n ""text"": ""-"",\n ""l...",nelson mandela eritrea,4,1,22,NaN,NaN,NaN


### Merging step 2
 - In the second merging step, we merge two candidates if their set of phrases heads are similar ( we apply lower similarity  threshold for the candidates of the same type)
 - merged candidates are stored in the dictionary what_merged2

In [16]:
import numpy as np
from scipy.spatial.distance import cosine

def merging_step2(candidate_list, model, what_merged2):
    
    indices_to_remove = set()
    
    for up_cand_id in tqdm(range(len(candidate_list))):     
        up_cand = candidate_list[up_cand_id]
        
        up_cand_mean_vec = phrase_heads_avg_vector(up_cand[2],model)
        
        for low_cand_id in range(up_cand_id+1,len(candidate_list)): 
            low_cand = candidate_list[low_cand_id]
            #print(f'for index {candidate_list.index(longer_cand)} checking the index {candidate_list.index(cand)}')
            #if candidate_list[longer_cand][1] == candidate_list[cand][1]:
                #print(f'matching "{longer_cand}" with "{cand}"')
            low_cand_mean_vec = phrase_heads_avg_vector(low_cand[2],model)
            similarity = 1-cosine(up_cand_mean_vec,low_cand_mean_vec)
            if up_cand[3] != 'misc': #up_cand[3] == low_cand[3]:
                if similarity >= 0.75:
                    if similarity < 0.77:
                        print(f'matching "{up_cand[0]}" with "{low_cand[0]}" of the same type with {similarity} sim') 
                    indices_to_remove.add(low_cand_id)
                    what_merged2[up_cand[0].lower()].append(low_cand)
                        

            else:
                if similarity >= 0.85:
                    if similarity < 0.87:
                        print(f'matching "{up_cand[0]}" with "{low_cand[0]}" of diff type with {similarity} sim') 
                    indices_to_remove.add(low_cand_id)
                    what_merged2[up_cand[0].lower()].append(low_cand)



    return indices_to_remove, what_merged2

def nans(shape,dtype=float):
    nans = np.empty(shape,dtype)
    nans.fill(np.nan)
    return nans

def phrase_heads_avg_vector(phrase_set,model):
    phrase_head_vectors = []
    for phrase_head in phrase_set:    
        try:
            phrase_head_vectors.append(model.wv[phrase_head.lower()])
        # KeyError is raised when the word is not in the vocabulary (should not happen, since the model is trained on the data)    
        except KeyError:
            phrase_head_vectors.append(nans(model.wv.vector_size))
            pass
        # we get an attribute error when we use pretrained model instead of self-trained    
        except AttributeError:
            try:
                phrase_head_vectors.append(model[phrase_head.lower()])
            # KeyError is raised when the word is not in the vocabulary (should not happen if we build our vocabulary on top of pretrained model)        
            except KeyError:
                phrase_head_vectors.append(nans(model.vector_size))
                pass
            
    #phrase_head_vectors = [model[phrase_head] for phrase_head in phrase_set]
    if len(phrase_head_vectors) != 0:
        return np.nanmean(phrase_head_vectors,axis=0)
    else: 
        return np.NaN

In [33]:
#model = load_pickle('moria_w2v_model')
#model = gensim.models.KeyedVectors.load_word2vec_format(r'C:\Users\nikodemicek\Dropbox (CBS)\Master thesis data\GoogleNews-vectors-negative300.bin.gz', binary=True)


#model = gensim.models.KeyedVectors.load_word2vec_format(r'C:\Users\nikodemicek\Dropbox (CBS)\Master thesis data\wiki-news-300d-1M-subword.vec')
model = load_pickle('moria_ft_model')

what_merged2 = defaultdict(list) # we map the merged candidates in the dictionary
event_cands_merged2 = event_cands_merged[:2000].copy()

indices_to_remove, what_merged2 = merging_step2(event_cands_merged2['candidates'], model, what_merged2)

event_cands_merged2 = merge_indices(event_cands_merged2, indices_to_remove)


  0%|                                                                                 | 3/2000 [00:00<10:32,  3.16it/s]

matching "fire" with "campfire" of diff type with 0.8651066422462463 sim
matching "fire" with "a firefighter" of diff type with 0.8628710508346558 sim


  0%|▏                                                                                | 4/2000 [00:01<10:14,  3.25it/s]

matching "turkey" with "turkiye" of the same type with 0.7526381611824036 sim
matching "migrants" with "scientists and immigrants" of diff type with 0.8521818518638611 sim


  0%|▏                                                                                | 5/2000 [00:01<10:13,  3.25it/s]

...

matching "napier" with "kherter" of the same type with 0.7563822269439697 sim
matching "napier" with "a new method" of the same type with 0.769660234451294 sim
matching "napier" with "rocket" of the same type with 0.7589207887649536 sim


 97%|███████████████████████████████████████████████████████████████████████████▌  | 1939/2000 [05:16<00:00, 77.17it/s]

matching "cuba" with "ukraine" of the same type with 0.7570672035217285 sim
matching "ukraine" with "petrou ralli" of the same type with 0.7507758736610413 sim
matching "ukraine" with "by melissa eddy" of the same type with 0.7547239065170288 sim
matching "ukraine" with "ulla jelpke" of the same type with 0.7631782293319702 sim
matching "ukraine" with "eight fire brigades" of the same type with 0.7630745768547058 sim
matching "rocket" with "petros giannakouris" of diff type with 0.853737473487854 sim
matching "rochester" with "nayapara" of the same type with 0.7644994258880615 sim
matching "nayapara" with "dorchester" of the same type with 0.7519998550415039 sim
matching "the greek military elements" with "instructions" of diff type with 0.8608605265617371 sim


100%|██████████████████████████████████████████████████████████████████████████████| 2000/2000 [05:16<00:00,  6.32it/s]

Initial amount of candidates: 2000
Amount of candidates: 821, after removing 1179 indices


In [34]:
what_merged2

defaultdict(list,
            {'greece': [('prime minister of greece',
               'minister',
               {'greece', 'minister'},
               'misc'),
              ('greeces', 'greeces', {'greeces'}, 'person-ne'),
              ('local anger as greece', 'anger', {'anger', 'greece'}, 'misc'),
              ('a calamity in greece',
               'calamity',
               {'calamity', 'greece'},
               'misc'),
              ('greecewho', 'greecewho', {'greecewho'}, 'loc-ne'),
              ('drowngreecethat',
               'drowngreecethat',
               {'drowngreecethat'},
               'misc'),
              (' enrichmentin greece',
               'enrichmentin',
               {'enrichmentin', 'greece'},
               'misc'),
              ('caritas greece', 'caritas', {'caritas', 'greece'}, 'group-ne'),
              ('unhcrgreece', 'unhcrgreece', {'unhcrgreece'}, 'group')],
             'refugees': [('refuge', 'refuge', {'refuge'}, 'misc'),
              

In [36]:
event_cands_merged2.head(20)

,candidates,cand_tags,cand_text,cand_len,cand_freq,string_len
0,"(greece, greece, {greece}, loc-ne)","[\n {\n ""id"": 1,\n ""text"": ""Greece"",\n ...",greece,1,18978,6
1,"(refugees, refugees, {refugees}, misc)","[\n {\n ""id"": 1,\n ""text"": ""refugees"",\...",refugees,1,10656,8
2,"(fire, fire, {fire}, misc)","[\n {\n ""id"": 1,\n ""text"": ""fire"",\n ...",fire,1,7477,4
3,"(turkey, turkey, {turkey}, loc-ne)","[\n {\n ""id"": 1,\n ""text"": ""Turkey"",\n ...",turkey,1,4986,6
4,"(migrants, migrants, {migrants}, misc)","[\n {\n ""id"": 1,\n ""text"": ""migrants"",\...",migrants,1,4666,8
5,"(lesbos, lesbos, {lesbos}, loc-ne)","[\n {\n ""id"": 1,\n ""text"": ""Lesbos"",\n ...",lesbos,1,3202,6
6,"(europe, europe, {europe}, loc-ne)","[\n {\n ""id"": 1,\n ""text"": ""Europe"",\n ...",europe,1,2516,6
7,"(euhomeaffairs, euhomeaffairs, {euhomeaffairs}...","[\n {\n ""id"": 1,\n ""text"": ""EUHomeAffai...",euhomeaffairs,1,2507,13
8,"(the greek islands, islands, {islands}, misc)","[\n {\n ""id"": 1,\n ""text"": ""the"",\n ...",the greek islands,3,2379,17
9,"(people, people, {people}, misc)","[\n {\n ""id"": 1,\n ""text"": ""people"",\n ...",people,1,2294,6


### Merging step 3
 - In the third merging step, we merge two candidates if their labeling phrases are similar 
 - merged candidates are stored in the dictionary what_merged3

In [37]:
what_merged3 = defaultdict(list)
event_cands_merged3 = event_cands_merged2.copy()

indices_to_remove, what_merged3 = merging_step3(event_cands_merged3, model, what_merged3)
event_cands_merged3 = merge_indices(event_cands_merged3, indices_to_remove)


what_merged3 = merge_transitively(what_merged3)
#what_merged3

100%|██████████████████████████████████████████████████████████████████████████████| 821/821 [00:00<00:00, 1076.03it/s]

Initial amount of candidates: 821
Amount of candidates: 757, after removing 64 indices


In [40]:
what_merged3

defaultdict(list,
            {'the greek islands': [('greek authorities',
               'authorities',
               {'authorities'},
               'misc'),
              ('crowded unsanitary conditions',
               'conditions',
               {'conditions'},
               'misc'),
              ('greek police', 'police', {'police'}, 'misc'),
              ('a greek immigrant', 'immigrant', {'immigrant'}, 'misc'),
              ('greek waters', 'waters', {'waters'}, 'misc'),
              ('greek officials', 'officials', {'officials'}, 'misc'),
              ('greek police move migrants',
               'move',
               {'migrants', 'move', 'police'},
               'misc'),
              ('greek soldiers', 'soldiers', {'soldiers'}, 'misc'),
              ('greek territory', 'territory', {'territory'}, 'misc'),
              ('greek pm promises', 'promises', {'pm', 'promises'}, 'misc'),
              ('greek govt', 'govt', {'govt'}, 'misc'),
              ('greek shephe

 ### Merging step 4
 - In the third merging step, we merge two candidates if their compound nouns are similar 
 - merged candidates are stored in the dictionary what_merged4

In [41]:
what_merged4 = defaultdict(list)
event_cands_merged4 = event_cands_merged3.copy()

indices_to_remove, what_merged4 = merging_step4(event_cands_merged4, model, what_merged4)
event_cands_merged4 = merge_indices(event_cands_merged4, indices_to_remove)

what_merged4 = merge_transitively(what_merged4)
#what_merged4

100%|████████████████████████████████████████████████████████████████████████████████| 757/757 [00:49<00:00, 15.21it/s]

Initial amount of candidates: 757
Amount of candidates: 163, after removing 594 indices


## Merge what_merged dictionaries

In [42]:
def merge_dicts(cands, wm2, wm3, wm4):
    # start with empty dictionary, the dictionary keys will be as many as candidates after 4th merging step
    merged = defaultdict(list)
    for cand in cands['cand_text']: #iterate over all cands
        # key's list of values will first contain all values from the same keys in the three merged dataframes
        merged[cand].extend(wm4[cand]) 
        merged[cand].extend(wm3[cand])
        merged[cand].extend(wm2[cand])
        
        # the values of the cand from 4th merging step can be keys in merging steps 3 & 2 
        for merged_cand3 in wm4[cand]: #iterate over all values in the list
            merged[cand].extend(wm3[merged_cand3[0]])
            merged[cand].extend(wm2[merged_cand3[0]])
            
            # the values of the cand from 3th merging step can be keys in merging step 2 
            for merged_cand2 in wm3[merged_cand3[0]]:
                merged[cand].extend(wm2[merged_cand2[0]])
            
    return merged

In [43]:
merged_dict = merge_dicts(event_cands_merged4, what_merged2, what_merged3, what_merged4)

In [44]:
for key,values in merged_dict.items():
    print(f'\n{key}: {set([val[0] for val in values])}')
event_cands_merged4.head(20)


greece: {'greekcitytimes', 'greek', 'prime minister of greece', 'a calamity in greece', 'greecewho', 'grenfell', 'greeces', 'drowngreecethat', ' enrichmentin greece', 'caritas greece', 'greekreporter', 'local anger as greece', 'greeks', 'unhcrgreece', 'greekisland'}

refugees: {'an asylum seeker', 'asylum seekers', 'constantinople', 'macedonians', 'a refugee', 'mercenaries', 'the dangers', 'refuge', 'failed asylum seekers', 'an illegalrefugee', 'therefugees', 'karen people', 'rohingya refugee camps unsays', 'the adventurers', 'rise in refugees', 'iraqi refugees tired', 'the inhabitants', '6refugees', 'refugeesgr', '90krefugees', 'people', 'refugees souls', 'few answers for refugees', 'iranian refugee omid masoumali'}

fire: {'al jazeera', 'samos', 'mytilene', 'mitillini island midilli', 'a small german firm', 'leros', 'our website', 'voices', 'high alert', 'the journey', 'ahval', 'greek refugee pushback campaign', 'many posted on amazon', 'croatia', 'mission in aegean sea', 'speedboat

,candidates,cand_tags,cand_text,cand_len,cand_freq,string_len
0,"(greece, greece, {greece}, loc-ne)","[\n {\n ""id"": 1,\n ""text"": ""Greece"",\n ...",greece,1,18978,6
1,"(refugees, refugees, {refugees}, misc)","[\n {\n ""id"": 1,\n ""text"": ""refugees"",\...",refugees,1,10656,8
2,"(fire, fire, {fire}, misc)","[\n {\n ""id"": 1,\n ""text"": ""fire"",\n ...",fire,1,7477,4
3,"(turkey, turkey, {turkey}, loc-ne)","[\n {\n ""id"": 1,\n ""text"": ""Turkey"",\n ...",turkey,1,4986,6
4,"(migrants, migrants, {migrants}, misc)","[\n {\n ""id"": 1,\n ""text"": ""migrants"",\...",migrants,1,4666,8
5,"(europe, europe, {europe}, loc-ne)","[\n {\n ""id"": 1,\n ""text"": ""Europe"",\n ...",europe,1,2516,6
6,"(euhomeaffairs, euhomeaffairs, {euhomeaffairs}...","[\n {\n ""id"": 1,\n ""text"": ""EUHomeAffai...",euhomeaffairs,1,2507,13
7,"(fr bau und heimat, fr, {heimat, fr}, misc)","[\n {\n ""id"": 1,\n ""text"": ""für"",\n ...",fr bau und heimat,4,1819,17
8,"(germany, germany, {germany}, loc-ne)","[\n {\n ""id"": 1,\n ""text"": ""Germany"",\n...",germany,1,1716,7
9,"(a humanitarian catastrophe, catastrophe, {cat...","[\n {\n ""id"": 1,\n ""text"": ""a"",\n ""l...",a humanitarian catastrophe,3,1602,26


In [19]:
#pickle_file('moria_short_cands_merged',event_cands_merged4)
pickle_file('moria_short_whatmerged2',what_merged2)
pickle_file('moria_short_whatmerged3',what_merged3)
pickle_file('moria_short_whatmerged4',what_merged4)

pickle_file('moria_short_cands_merged',event_cands_merged4)
pickle_file('moria_merged_dict',merged_dict)

#event_cands_merged4 = load_pickle("moria_short_cands_merged")


## Additional merging attempts 1: Using Affinity propagation clustering

In [ ]:
merged['greece']

In [ ]:
w2v_model = load_pickle('moria_ft_model')

In [ ]:
w2v_model.wv['greece']

In [ ]:
from sklearn.cluster import AffinityPropagation
import numpy as np


X = np.array([model.wv[word[1]] for word in event_cands_merged4['candidates']])


from sklearn.preprocessing import StandardScaler

scaled_x = StandardScaler().fit_transform(X)

from sklearn.decomposition import PCA
pca = PCA(n_components=2)
pca_x = pca.fit_transform(X)


clustering = AffinityPropagation(max_iter=500,random_state=42, verbose=True).fit(pca_x)
print(clustering)

In [ ]:
import pandas as pd
event_cands_merged4['labels'] = pd.Series(clustering.labels_)
event_cands_merged4['labels'].value_counts()

## Comparison of models in Merging step 2

In [30]:
#FIRST VARIATION FASTTEXT MODEL TRAINED ON MORIA DATA, THRESHOLDS 0.7 for same types and 0.8 otherwise

model1 = load_pickle('moria_fasttext_model')

what_merged2_1 = defaultdict(list) # we map the merged candidates in the dictionary
event_cands_merged2_1 = event_cands_merged.copy()

indices_to_remove, what_merged2_1 = merging_step2(event_cands_merged2_1['candidates'], model1, what_merged2_1)

event_cands_merged2_1 = merge_indices(event_cands_merged2_1, indices_to_remove)

matching "petersweden" with "a greco  swede" of the same type with 0.70261150598526 sim
matching "petersweden" with "voden" of diff type with 0.8057663440704346 sim


 37%|██████████████████████████▏                                           | 10827/28941 [15:53:45<22:23:11,  4.45s/it]

matching "herself on fire" with "the bullies in fire" of the same type with 0.9556370973587036 sim
matching "herself on fire" with "ambulances and fire" of the same type with 0.9501142501831055 sim
matching "herself on fire" with "velika kladua on fire" of the same type with 0.9653425216674805 sim
matching "herself on fire" with "methuenfire1691" of diff type with 0.8125804662704468 sim
matching "herself on fire" with "the cross fire naha" of the same type with 0.9626156091690063 sim
matching "herself on fire" with "firefighers" of the same type with 0.8559474945068359 sim
matching "herself on fire" with "appropriate to fire someone" of the same type with 0.9300042986869812 sim
matching "herself on fire" with "sutherland shire  rnp" of diff type with 0.8000339865684509 sim
matching "herself on fire" with "nashvillefiredept" of the same type with 0.7907846570014954 sim
matching "herself on fire" with "migrant boat catch fire" of the same type with 0.8617865443229675 sim
matching "hersel

matching "herself on fire" with "south ayrshire" of diff type with 0.8102748394012451 sim
matching "herself on fire" with "camp  fiweh" of the same type with 0.7016478180885315 sim
matching "herself on fire" with "scarlet begonias  fire" of the same type with 0.9519288539886475 sim
matching "herself on fire" with "slaithwaite fire service collections" of the same type with 0.8896652460098267 sim
matching "herself on fire" with "firecrotone" of the same type with 0.8888998031616211 sim
matching "herself on fire" with "moria fire archyde" of the same type with 0.9491018652915955 sim
matching "herself on fire" with "firefigthers" of the same type with 0.8497673273086548 sim
matching "herself on fire" with "laconia fire suspicious" of the same type with 0.9499112963676453 sim
matching "herself on fire" with "fire teargasfire" of the same type with 0.8224470615386963 sim
matching "herself on fire" with "welcome fire refugee cupcakes" of the same type with 0.9505058526992798 sim
matching "he

 37%|██████████████████████████▏                                           | 10828/28941 [15:53:49<22:35:12,  4.49s/it]

...

matching "20200701 cbc" with "total   126" of the same type with 0.748656153678894 sim
matching "20200701 cbc" with "bedtime2  8" of the same type with 0.7815717458724976 sim
matching "20200701 cbc" with "1993 y" of the same type with 0.7096225619316101 sim
matching "20200701 cbc" with "dubpl8  007" of the same type with 0.7006160020828247 sim


 38%|██████████████████████████▎                                           | 10869/28941 [15:56:29<22:38:08,  4.51s/it]

matching "the gender asymmetry" with "asymptomatic" of the same type with 0.7863169312477112 sim
matching "the gender asymmetry" with "an entire hybrid" of the same type with 0.7132734656333923 sim


 38%|██████████████████████████▎                                           | 10869/28941 [15:56:31<26:30:26,  5.28s/it]


KeyboardInterrupt: 

In [8]:
#SECOND VARIATION FASTTEXT PRETRAINED MODEL, THRESHOLDS 0.7 for same types and 0.8 otherwise

model2 = gensim.models.KeyedVectors.load_word2vec_format(r'C:\Users\nikodemicek\Dropbox (CBS)\Master thesis data\wiki-news-300d-1M-subword.vec')

what_merged2_2 = defaultdict(list) # we map the merged candidates in the dictionary
event_cands_merged2_2 = event_cands_merged.copy()

indices_to_remove, what_merged2_2 = merging_step2(event_cands_merged2_2['candidates'], model2, what_merged2_2)

event_cands_merged2_2 = merge_indices(event_cands_merged2_2, indices_to_remove)

  0%|                                                                                         | 0/2063 [00:00<?, ?it/s]C:\Users\nikodemicek\Documents\GitHub\refugee_project\merging_steps.py:104: RuntimeWarning: Mean of empty slice
  return np.nanmean(phrase_head_vectors,axis=0)


matching "greece" with "athens" of the same type with 0.7315918803215027 sim
matching "greece" with "italy" of the same type with 0.7035752534866333 sim
matching "greece" with "prime minister of greece" of diff type with 0.8634316921234131 sim
matching "greece" with "cyprus" of the same type with 0.7422357797622681 sim
matching "greece" with "bulgaria" of the same type with 0.7303168177604675 sim
matching "greece" with "macedonia" of the same type with 0.765284538269043 sim
matching "greece" with "albania" of the same type with 0.7289749383926392 sim


  0%|                                                                                 | 1/2063 [00:00<13:05,  2.62it/s]

matching "greece" with "a calamity in greece" of diff type with 0.8892802596092224 sim
matching "greece" with " enrichmentin greece" of diff type with 0.9999999972355171 sim
matching "greece" with "caritas greece" of diff type with 0.8874796032905579 sim
matching "refugees" with "migrants" of the same type with 0.7617825269699097 sim
matching "refugees" with "immigrants" of the same type with 0.7510162591934204 sim
matching "refugees" with "ministeriejenv thousands of refugees" of the same type with 0.885500696692674 sim
matching "refugees" with "a refugee" of the same type with 0.8043560981750488 sim


  0%|                                                                                 | 2/2063 [00:00<12:12,  2.82it/s]

...

matching "george bizos" with "chris howard" of the same type with 0.7667654655427973 sim


 95%|██████████████████████████████████████████████████████████████████████████▍   | 1968/2063 [06:17<00:02, 46.71it/s]

matching "i e" with "caliban s" of the same type with 0.7114444331370876 sim


100%|██████████████████████████████████████████████████████████████████████████████| 2063/2063 [06:18<00:00,  5.45it/s]

Initial amount of candidates: 2063
Amount of candidates: 1348, after removing 715 indices


In [ ]:
#THIRD VARIATION W2V PRETRAINED MODEL, THRESHOLDS 0.7 for same types and 0.8 otherwise

model3 = gensim.models.KeyedVectors.load_word2vec_format(r'C:\Users\nikodemicek\Dropbox (CBS)\Master thesis data\GoogleNews-vectors-negative300.bin.gz', binary=True)

what_merged2_3 = defaultdict(list) # we map the merged candidates in the dictionary
event_cands_merged2_3 = event_cands_merged.copy()

indices_to_remove, what_merged2_3 = merging_step2(event_cands_merged2_3['candidates'], model3, what_merged2_3)

event_cands_merged2_3 = merge_indices(event_cands_merged2_3, indices_to_remove)

In [ ]:
#FOURTH VARIATION W2V MODEL TRAINED ON MORIA DATA, THRESHOLDS 0.7 for same types and 0.8 otherwise

model4 = load_pickle('moria_w2v_model')

what_merged2_4 = defaultdict(list) # we map the merged candidates in the dictionary
event_cands_merged2_4 = event_cands_merged.copy()

indices_to_remove, what_merged2_4 = merging_step2(event_cands_merged2_4['candidates'], model4, what_merged2_4)

event_cands_merged2_4 = merge_indices(event_cands_merged2_4, indices_to_remove)

In [11]:
pickle_file('moria_wm2_1',what_merged2_1)
pickle_file('moria_wm2_2',what_merged2_2)
pickle_file('moria_wm2_3',what_merged2_3)
pickle_file('moria_wm2_4',what_merged2_4)

pickle_file('moria_after2_1', event_cands_merged2_1)
pickle_file('moria_after2_2', event_cands_merged2_2)
pickle_file('moria_after2_3', event_cands_merged2_3)
pickle_file('moria_after2_4', event_cands_merged2_4)

In [ ]:
what_merged2_1 = load_pickle('moria_wm2_1')
what_merged2_2 = load_pickle('moria_wm2_2')
what_merged2_3 = load_pickle('moria_wm2_3')
what_merged2_4 = load_pickle('moria_wm2_4')

event_cands_merged2_1 = load_pickle('moria_after2_1')
event_cands_merged2_2 = load_pickle('moria_after2_2')
event_cands_merged2_3 = load_pickle('moria_after2_3')
event_cands_merged2_4 = load_pickle('moria_after2_4')

model1 = load_pickle('moria_fasttext_model')
print('1st model loaded')
model2 = gensim.models.KeyedVectors.load_word2vec_format(r'C:\Users\nikodemicek\Dropbox (CBS)\Master thesis data\wiki-news-300d-1M-subword.vec')
print('2nd model loaded')
model3 = gensim.models.KeyedVectors.load_word2vec_format(r'C:\Users\nikodemicek\Dropbox (CBS)\Master thesis data\GoogleNews-vectors-negative300.bin.gz', binary=True)
print('3rd model loaded')
model4 = load_pickle('moria_w2v_model')
print('4th model loaded')

## Comparison of models in Merging step 3

In [ ]:
what_merged3_1 = defaultdict(list)
event_cands_merged3_1 = event_cands_merged2_1.copy()

indices_to_remove, what_merged3_1 = merging_step3(event_cands_merged3_1, model1, what_merged3_1)
event_cands_merged3_1 = merge_indices(event_cands_merged3_1, indices_to_remove)


what_merged3_1 = merge_transitively(what_merged3_1)

In [13]:
what_merged3_2 = defaultdict(list)
event_cands_merged3_2 = event_cands_merged2_2.copy()

indices_to_remove, what_merged3_2 = merging_step3(event_cands_merged3_2, model2, what_merged3_2)
event_cands_merged3_2 = merge_indices(event_cands_merged3_2, indices_to_remove)


what_merged3_2 = merge_transitively(what_merged3_2)

100%|█████████████████████████████████████████████████████████████████████████████| 1348/1348 [00:05<00:00, 267.42it/s]

Initial amount of candidates: 1348
Amount of candidates: 1233, after removing 115 indices


In [14]:
what_merged3_3 = defaultdict(list)
event_cands_merged3_3 = event_cands_merged2_3.copy()

indices_to_remove, what_merged3_3 = merging_step3(event_cands_merged3_3, model3, what_merged3_3)
event_cands_merged3_3 = merge_indices(event_cands_merged3_3, indices_to_remove)


what_merged3_3 = merge_transitively(what_merged3_3)


100%|█████████████████████████████████████████████████████████████████████████████| 1715/1715 [00:10<00:00, 168.80it/s]

Initial amount of candidates: 1715
Amount of candidates: 1589, after removing 126 indices


In [15]:
what_merged3_4 = defaultdict(list)
event_cands_merged3_4 = event_cands_merged2_4.copy()

indices_to_remove, what_merged3_4 = merging_step3(event_cands_merged3_4, model4, what_merged3_4)
event_cands_merged3_4 = merge_indices(event_cands_merged3_4, indices_to_remove)


what_merged3_4 = merge_transitively(what_merged3_4)


100%|███████████████████████████████████████████████████████████████████████████████| 308/308 [00:00<00:00, 837.48it/s]

Initial amount of candidates: 308
Amount of candidates: 289, after removing 19 indices


## Comparison of models in Merging step 4

In [ ]:
what_merged4_1 = defaultdict(list)
event_cands_merged4_1 = event_cands_merged3_1.copy()

indices_to_remove, what_merged4_1 = merging_step4(event_cands_merged4_1, model1, what_merged4_1)
event_cands_merged4_1 = merge_indices(event_cands_merged4_1, indices_to_remove)

what_merged4_1 = merge_transitively(what_merged4_1)

In [17]:
what_merged4_2 = defaultdict(list)
event_cands_merged4_2 = event_cands_merged3_2.copy()

indices_to_remove, what_merged4_2 = merging_step4(event_cands_merged4_2, model2, what_merged4_2)
event_cands_merged4_2 = merge_indices(event_cands_merged4_2, indices_to_remove)

what_merged4_2 = merge_transitively(what_merged4_2)

100%|██████████████████████████████████████████████████████████████████████████████| 1233/1233 [04:52<00:00,  4.21it/s]


Initial amount of candidates: 1233
Amount of candidates: 305, after removing 928 indices


In [18]:
what_merged4_3 = defaultdict(list)
event_cands_merged4_3 = event_cands_merged3_3.copy()

indices_to_remove, what_merged4_3 = merging_step4(event_cands_merged4_3, model3, what_merged4_3)
event_cands_merged4_3 = merge_indices(event_cands_merged4_3, indices_to_remove)

what_merged4_3 = merge_transitively(what_merged4_3)

100%|██████████████████████████████████████████████████████████████████████████████| 1589/1589 [09:17<00:00,  2.85it/s]


Initial amount of candidates: 1589
Amount of candidates: 452, after removing 1137 indices


In [19]:
what_merged4_4 = defaultdict(list)
event_cands_merged4_4 = event_cands_merged3_4.copy()

indices_to_remove, what_merged4_4 = merging_step4(event_cands_merged4_4, model4, what_merged4_4)
event_cands_merged4_4 = merge_indices(event_cands_merged4_4, indices_to_remove)

what_merged4_4 = merge_transitively(what_merged4_4)


100%|████████████████████████████████████████████████████████████████████████████████| 289/289 [00:10<00:00, 27.56it/s]

Initial amount of candidates: 289
Amount of candidates: 133, after removing 156 indices


## Final candidates and merged dictionaries

In [20]:
merged_dict1 = merge_dicts(event_cands_merged4_1, what_merged2_1, what_merged3_1, what_merged4_1)
#merged_dict2 = merge_dicts(event_cands_merged4_2, what_merged2_2, what_merged3_2, what_merged4_2)
#merged_dict3 = merge_dicts(event_cands_merged4_3, what_merged2_3, what_merged3_3, what_merged4_3)
#merged_dict4 = merge_dicts(event_cands_merged4_4, what_merged2_4, what_merged3_4, what_merged4_4)

In [ ]:
for key,values in merged_dict1.items():
    print(f'\n{key}: {set([val[0] for val in values])}')
event_cands_merged4_1.head(20)

In [20]:
for key,values in merged_dict2.items():
    print(f'\n{key}: {set([val[0] for val in values])}')
event_cands_merged4_2.head(20)

NameError: name 'merged_dict2' is not defined

In [ ]:
for key,values in merged_dict3.items():
    print(f'\n{key}: {set([val[0] for val in values])}')
event_cands_merged4_3.head(20)

In [ ]:
for key,values in merged_dict4.items():
    print(f'\n{key}: {set([val[0] for val in values])}')
event_cands_merged4_4.head(20)